In [1]:
import transformers

#Set to avoid warning messages.
transformers.logging.set_verbosity_error()

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 06.02 Running the standard NER Pipeline

In [2]:
from transformers import pipeline

input_text="Sam went to California on the 23rd of August. \
There, he visited Google headquarters with John Smith and bought a cap for $23"

basic_ner = pipeline("ner")

basic_ner(input_text)


2025-06-26 18:58:44.907041: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-26 18:58:45.173628: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-26 18:58:45.323398: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750964325.620549    8452 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750964325.717265    8452 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750964326.317063    8452 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

[{'entity': 'I-PER',
  'score': np.float32(0.99887806),
  'index': 1,
  'word': 'Sam',
  'start': 0,
  'end': 3},
 {'entity': 'I-LOC',
  'score': np.float32(0.99972683),
  'index': 4,
  'word': 'California',
  'start': 12,
  'end': 22},
 {'entity': 'I-ORG',
  'score': np.float32(0.9960085),
  'index': 15,
  'word': 'Google',
  'start': 64,
  'end': 70},
 {'entity': 'I-PER',
  'score': np.float32(0.99891376),
  'index': 18,
  'word': 'John',
  'start': 89,
  'end': 93},
 {'entity': 'I-PER',
  'score': np.float32(0.99921584),
  'index': 19,
  'word': 'Smith',
  'start': 94,
  'end': 99}]

## 06.03. Understanding the model architecture

In [3]:
#Print model architecture
print(basic_ner.model)


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 1024, padding_idx=0)
      (position_embeddings): Embedding(512, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-23): 24 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024

## 06.04 Reviewing model configuration

In [4]:
print(basic_ner.model.config)


BertConfig {
  "_num_labels": 9,
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "O",
    "1": "B-MISC",
    "2": "I-MISC",
    "3": "B-PER",
    "4": "I-PER",
    "5": "B-ORG",
    "6": "I-ORG",
    "7": "B-LOC",
    "8": "I-LOC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "B-LOC": 7,
    "B-MISC": 1,
    "B-ORG": 5,
    "B-PER": 3,
    "I-LOC": 8,
    "I-MISC": 2,
    "I-ORG": 6,
    "I-PER": 4,
    "O": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "positi

## 06.05. Using a Custom Model and tokenizer

In [6]:
from transformers import AutoTokenizer, TFAutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/camembert-ner-with-dates", 
                                          from_pt=True)

model = TFAutoModelForTokenClassification.from_pretrained("Jean-Baptiste/camembert-ner-with-dates",
                                                          from_pt=True)

print(model.config.id2label)

{0: 'O', 1: 'I-LOC', 2: 'I-PER', 3: 'I-MISC', 4: 'I-ORG', 5: 'I-DATE'}


In [7]:
#Prediction
enhanced_ner = pipeline('ner', 
                        model=model, 
                        tokenizer=tokenizer, 
                        aggregation_strategy="simple")
enhanced_ner(input_text)

[{'entity_group': 'PER',
  'score': np.float32(0.97762144),
  'word': 'Sam',
  'start': np.int32(0),
  'end': np.int32(3)},
 {'entity_group': 'LOC',
  'score': np.float32(0.9936407),
  'word': 'California',
  'start': np.int32(11),
  'end': np.int32(22)},
 {'entity_group': 'DATE',
  'score': np.float32(0.9235599),
  'word': 'August',
  'start': np.int32(37),
  'end': np.int32(44)},
 {'entity_group': 'ORG',
  'score': np.float32(0.57216746),
  'word': 'Google',
  'start': np.int32(63),
  'end': np.int32(70)},
 {'entity_group': 'PER',
  'score': np.float32(0.9938346),
  'word': 'John Smith',
  'start': np.int32(88),
  'end': np.int32(99)},
 {'entity_group': 'DATE',
  'score': np.float32(0.64064384),
  'word': '23',
  'start': np.int32(122),
  'end': np.int32(124)}]